### 기본 세팅

In [1]:
import time
from datetime import date, timedelta
import pandas as pd

from selenium import webdriver
from bs4 import BeautifulSoup

In [2]:
# 어제 날짜 저장
def get_yesterday():
    today     = date.today()             # 오늘 날짜
    yesterday = today - timedelta(1)     # 어제 날짜

    year = str(yesterday.year)
    mon = str(yesterday.month)
    if int(mon) < 10:
        mon = '0' + str(mon)
    day = str(yesterday.day)
    if int(day) < 10:
        day = '0' + str(day)
        
    return year+mon+day

### 다음 뉴스 크롤링

In [3]:
def get_daumnews_info(keyword):
    
    # 크롤링 정보(크롤링 날짜, 카테고리)
    yesterday = get_yesterday()    
    categories = ['finance', 'industry', 'autos', 'stock', 'stock/market', 'stock/publicnotice', 'stock/others', 'consumer']
    
    list_headline_concat    = []     # 모든 카테고리 제목
    list_address_concat     = []     # 모든 카테고리 주소 리스트
    list_comment_num_concat = []     # 모든 카테고리 댓글개수 리스트
    comment_headline_concat = []     # 모든 카테고리 댓글모을 제목 리스트
    comment_comment_concat  = []     # 모든 카테고리 제목별 댓글 리스트    
    
    
    
    # 드라이버 실행
    driver = webdriver.Chrome('chromedriver.exe')
    driver.implicitly_wait(3)
    sleeptime = 0.5     # 프로세스 일시정지 시간
    
    
    
    # 카테고리를 돌며 크롤링
    for category in categories:
        page = 0
        list_headline = []     # 제목 리스트
        list_address  = []     # 주소 리스트
        drop_address  = []     # 키워드 미포함 주소 리스트
        top_headline = ''              # 페이지내 맨위 뉴스 제목
        top_headline_flag = False     # 다음 페이지 맨위 뉴스제목이 기존 페이지 맨위 뉴스제목과 다름  

        
        
        # 제목 및 주소 크롤링
        while True:
            try:
                # 페이지 이동
                page += 1
                driver.get(f'https://news.daum.net/breakingnews/economic/{category}?page={page}&regDate={yesterday}')
                time.sleep(sleeptime)        

                
                
                # 제목 크롤링
                current = driver.find_elements_by_css_selector('#mArticle > div > ul > li > div > strong > a')
                cnt = 0
                drop_address  = []     # 키워드 미포함 주소 리스트
                for headline in current:
                    # 다음 페이지 맨위 뉴스제목이 기존 페이지 맨위 뉴스제목과 같으면 해당 카테고리 종료
                    if cnt == 0 and headline.text == top_headline:
                        top_headline_flag = True
                        break
                    
                    # 페이지내 맨위 뉴스제목 저장
                    if cnt == 0:
                        top_headline = headline.text
                    
                    # 키워드 있는 제목만 저장
                    if keyword in headline.text:
                        list_headline.append(headline.text)

                    # 키워드 없는 제목 인덱스 저장
                    if keyword not in headline.text:
                        drop_address.append(cnt)
                    cnt += 1

                # 다음 페이지 맨위 뉴스제목이 기존 페이지 맨위 뉴스제목과 같으면 해당 카테고리 종료
                if top_headline_flag == True:
                    break

                    
                    
                # 뉴스주소 크롤링
                cnt = 0
                for address in current:
                    # 키워드 없는 제목인 경우 주소 저장 안함
                    if cnt in drop_address:
                        cnt += 1
                        continue

                    # 키워드 있는 제목만 주소 저장
                    if address.get_attribute("href") not in list_address:
                        list_address.append(address.get_attribute("href"))
                    cnt += 1
            except:
                pass



        cnt = 0
        list_comment     = []     # 댓글 리스트
        list_comment_num = []     # 댓글개수 리스트

        # 키워드 포함한 게시글 주소를 돌며 크롤링
        for url in list_address:
            try:
                # 페이지 이동
                cnt += 1
                driver.get(url)
                time.sleep(sleeptime)        

                
                
                # 댓글개수 크롤링
                current = driver.find_elements_by_css_selector('#alexCounter > span')
                for num in current:
                    list_comment_num.append(num.text)

                    
                    
                # 댓글 있는 경우
                if list_comment_num[cnt-1] != '0':
                    # 댓글 더보기 클릭  
                    try:
                        while driver.find_element_by_xpath('//*[@id="alex-area"]/div/div/div/div[3]/div[3]/button').text != '':
                            driver.find_element_by_xpath('//*[@id="alex-area"]/div/div/div/div[3]/div[3]/button').click()
                            time.sleep(0.5)
                    except:
                        pass

                    # 댓글 크롤링
                    soup = BeautifulSoup(driver.page_source, 'html.parser')
                    comments = soup.find_all('p', {'class': 'desc_txt font_size_17'})
                    cur_comment = []
                    for comment in comments:
                        cur_comment.append(comment.text)
                    list_comment.append(cur_comment)     

                    
                    
                # 댓글 없는 경우
                if list_comment_num[cnt-1] == '0':
                    cur_comment = []
                    list_comment.append(cur_comment)
            except:
                pass    



        comment_headline = []     # 댓글모을 제목 리스트
        comment_comment  = []     # 제목별 댓글 리스트

        # 제목별 댓글 모음
        for i in range(len(list_comment_num)):
            cur_headline = list_headline[i]

            # 댓글 없는 경우 해당 안됨
            if int(list_comment_num[i]) == 0:
                continue

            for j in range(len(list_comment[i])):
                cur_comment = list_comment[i][j]

                comment_headline.append(cur_headline)
                comment_comment.append(cur_comment)
                
                
                
        # 카테고리 통합
        list_headline_concat.extend(list_headline)     
        list_address_concat.extend(list_address)     
        list_comment_num_concat.extend(list_comment_num)     
        comment_headline_concat.extend(comment_headline)
        comment_comment_concat.extend(comment_comment)
    driver.close()
        

        
    daumnews_info = pd.DataFrame({'제목': list_headline_concat, '주소': list_address_concat, '댓글수': list_comment_num_concat})
    daumnews_comment = pd.DataFrame({'제목': comment_headline_concat, '댓글': comment_comment_concat})        
    return daumnews_info, daumnews_comment

### 크롤링 예시

In [4]:
daumnews_HMM_info, daumnews_HMM_comment = get_daumnews_info('HMM')

In [5]:
daumnews_HMM_info

,제목,주소,댓글수
0,파업 일단 보류 HMM..산업은행 침묵 깨고 해결 나설까,https://v.daum.net/v/20210826062703979,10
1,HMM 노사 벼랑 끝 대치..누굴 탓하랴,https://v.daum.net/v/20210826072005611,31
2,"사측·노조 '9월1일 재교섭'..HMM, 극적타결 이끌어 낼까",https://v.daum.net/v/20210826051007130,3
3,"HMM 파업 보류.. 해원노조 ""9월 1일 재교섭""",https://v.daum.net/v/20210826030431056,0


In [6]:
daumnews_HMM_comment

,제목,댓글
0,파업 일단 보류 HMM..산업은행 침묵 깨고 해결 나설까,개입해라 국민즐 세금으로 지원한 거니 임금을 올려주든 안 주든 개입해야지 왜 안하냐...
1,파업 일단 보류 HMM..산업은행 침묵 깨고 해결 나설까,"회사 힘들때 임금동결하고 묵묵하게 일하다가 몇년만에 역대최고의 실적이 나오니까,\n..."
2,파업 일단 보류 HMM..산업은행 침묵 깨고 해결 나설까,산업은행 총수 잘라라\n나라 경제 말아먹을 작자다\n\n8년간 고통분담한 사원들을 ...
3,파업 일단 보류 HMM..산업은행 침묵 깨고 해결 나설까,저봐 뭣도 모르고 혈세 혈세 하자나.\n제발 좀 가져 가라고 \n채권 갚는다는데 왜...
4,파업 일단 보류 HMM..산업은행 침묵 깨고 해결 나설까,주가 5만원에만 매각해도 전량회수하고도 이익이 따블일텐대\n주가부양을 안하네 헐
5,파업 일단 보류 HMM..산업은행 침묵 깨고 해결 나설까,공적자금은 국민들의 혈세다. 무조건 근로자들한테 인내하라고 할수없지만 혈세를갚으면...
6,파업 일단 보류 HMM..산업은행 침묵 깨고 해결 나설까,국민 세금 투자해서 살려놓으니 상황이 잠시 좋아 졌다고 협박하내.
7,파업 일단 보류 HMM..산업은행 침묵 깨고 해결 나설까,산은에 돈은 갚고 이야기해라....
8,파업 일단 보류 HMM..산업은행 침묵 깨고 해결 나설까,2조 4천억 수익은 벌은게 아니고 엿바꿔 먹은건가?? \n그돈을 수익으로 잡으면 6...
9,파업 일단 보류 HMM..산업은행 침묵 깨고 해결 나설까,이젠좀 나섰으면?
